In [1]:
%%writefile wgs_ecef.py
"""
Convert between WGS and ECEF coordinates

A python-numba implementation

Algorithms from D. Rose:
"Converting between Earth-Centered, Earth Fixed and Geodetic Coordinates"
danceswithcode.net/engineeringnotes/geodetic_to_ecef/geodetic_to_ecef.html


# WGS (World Geodetic System)
wikipedia.org/wiki/World_Geodetic_System
specifies longtide (degrees), latitude (degrees), and
altitude (meters) above "sea level"


# ECEF (Earth-Centered, Earth-Fixed)
wikipedia.org/wiki/ECEF
specifies three cartesian coordinates x, y, z (meters).


# Example
> check()
passed all checks
> ecef_to_wgs(-4e6, 3e6, 3.9467e6)
(0.6714761701513846, 2.498091544796509, 61.58916153024969)
> wgs_to_ecef(*ecef_to_wgs(-4e6, 3e6, 3.9467e6))
(-4000000.0, 3000000.0, 3946700.0000000005)

# See this notebook for more: TODO

"""
import math

import numpy
import numba



# WGS-84 parameters: equatorial radius, squared eccentricity

WGS_A = 6378137.0
WGS_E2 = 6.6943799901377997e-3


# The "Google Smartphone Decimeter Challenge" uses degrees; convert.

RAD_TO_DEG = 180 / math.pi
DEG_TO_RAD = math.pi / 180



# Utilities

def wgs_to_ecef(lat, lng, alt):
    """ Return ECEF coordinates x, y, z.

        Arguments:
            lat, lng, alt:
                WGS coordinates;
                latitude in degrees,
                longitude in degrees,
                altitude in meters.
                Optimized for float64 ndarrays of shape (n,),
                but also accepts scalars.

        Returns:
            x, y, z:
                ECEF coordinates in meters.
                All have the same shape as inputs.
    """
    lat = numpy.asarray(lat)
    lng = numpy.asarray(lng)
    alt = numpy.asarray(alt)
    shape = lat.shape
    assert shape == lng.shape == alt.shape

    if shape == ():
        return wgs_to_ecef_one(lat.item(), lng.item(), alt.item())

    return wgs_to_ecef_vec(lat, lng, alt)


def ecef_to_wgs(x, y, z):
    """ Return WGS coordinates lat, lng, alt.

        Arguments:
            x, y, z:
                ECEF coordinates in meters.
                Optimized for float64 ndarrays of shape (n,),
                but also accepts scalars.

        Returns:
            lat, lng, alt:
                WGS coordinates;
                latitude in degrees,
                longitude in degrees,
                altitude in meters.
                All have the same shape as inputs.
    """
    x = numpy.asarray(x)
    y = numpy.asarray(y)
    z = numpy.asarray(z)
    shape = x.shape
    assert shape == y.shape == z.shape

    if shape == ():
        return ecef_to_wgs_one(x.item(), y.item(), z.item())

    return ecef_to_wgs_vec(x, y, z)



# Implementation

@numba.njit(
    (numba.float64,) * 3,
    fastmath={"arcp", "contract"})
def wgs_to_ecef_one(lat, lng, alt):
    """ Scalar implementation of wgs_to_ecef(...). """
    lat *= DEG_TO_RAD
    lng *= DEG_TO_RAD

    sinlat = numpy.sin(lat)
    coslat = numpy.cos(lat)
    n = WGS_A / (1 - WGS_E2*sinlat*sinlat)**0.5

    x = (n + alt)*coslat*numpy.cos(lng)
    y = (n + alt)*coslat*numpy.sin(lng)
    z = (n*(1 - WGS_E2) + alt) * sinlat
    return x, y, z


@numba.njit(
    (numba.float64,) * 3,
    fastmath={"arcp", "contract"})
def ecef_to_wgs_one(x, y, z):
    """ Scalar implementation of ecef_to_wgs(...). """
    norm2 = 1 / (x*x + y*y + z*z)
    norm = norm2 ** 0.5

    waist = (x*x + y*y)**0.5
    sin2 = z*z * norm2
    cos2 = (x*x + y*y) * norm2

    a1 = WGS_A*WGS_E2
    u = a1*norm
    v = 0.5*WGS_E2 - 2.5*a1*norm

    if cos2 > 0.3:
        sin_ = abs(z)*norm*(1 + norm*cos2*a1*(1 + u + sin2*v))
        cos_ = (1 - sin_*sin_)**0.5
        g = 1 / (1 - WGS_E2*sin_*sin_)
        theta = numpy.arcsin(sin_)
    else:
        cos_ = waist*norm*(1 - norm*sin2*a1*(1 + 0.5*WGS_E2 - u - cos2*v))
        sin_ = (1 - cos_*cos_)**0.5
        g = 1 / (1 - WGS_E2*(1 - cos_*cos_))
        theta = numpy.arccos(cos_)

    rg = WGS_A * g**0.5
    rf = (1 - WGS_E2)*rg
    u = waist - cos_*rg
    v = abs(z) - rf*sin_
    f = cos_*u + sin_*v
    m = cos_*v - sin_*u
    p = m / (rf*g + f)

    lat = (theta + p) * numpy.copysign(1, z) * RAD_TO_DEG
    lng = numpy.arctan2(y, x) * RAD_TO_DEG
    alt = f + m*p*0.5
    return lat, lng, alt


@numba.guvectorize(
    [(numba.float64[:],) * 6],
    "(n),(n),(n)->(n),(n),(n)",
    fastmath={"arcp", "contract"})
def wgs_to_ecef_vec(lat, lng, alt, x, y, z):
    """ Vector implementation of ecef_to_wgs(...). """
    for i in range(lat.shape[0]):
        x[i], y[i], z[i] = wgs_to_ecef_one(lat[i], lng[i], alt[i])


@numba.guvectorize(
    [(numba.float64[:],) * 6],
    "(n),(n),(n)->(n),(n),(n)",
    fastmath={"arcp", "contract"})
def ecef_to_wgs_vec(x, y, z, lat, lng, alt):
    """ Vector implementation of ecef_to_wgs(...). """
    for i in range(x.shape[0]):
        lat[i], lng[i], alt[i] = ecef_to_wgs_one(x[i], y[i], z[i])



# Testing

def check():
    """ Run all checks. """
    check_ecef_to_wgs(ecef_to_wgs)
    check_wgs_to_ecef(wgs_to_ecef)
    check_both(ecef_to_wgs, wgs_to_ecef)
    print("passed all checks")


def check_ecef_to_wgs(func_check):
    # check some examples
    assert func_check(WGS_A, 0, 0) == (0, 0, 0)

    points = [
        (1e6, 1e6, 1e6),
        (-1e6, 1e6, 1e6),
        (1e6, -1e6, 1e6),
        (1e6, 1e6, -1e6),
    ]

    for point in points:
        numpy.testing.assert_allclose(
            func_check(*point), ecef_to_wgs_ref(*point))

    rng = numpy.random.Generator(numpy.random.Philox(1234))

    for _ in range(10**3):
        point = rng.normal(scale=6e6, size=3)
        numpy.testing.assert_allclose(
            func_check(*point), ecef_to_wgs_ref(*point))

    # check vector-scalar equality
    points = rng.normal(scale=6e6, size=(3, 10**3))
    lats, lngs, alts = func_check(*points)

    for lat, lng, alt, point in zip(lats, lngs, alts, points.T):
        numpy.testing.assert_array_equal((lat, lng, alt), func_check(*point))


def check_wgs_to_ecef(func_check):
    # check some examples
    assert func_check(0, 0, 0) == (WGS_A, 0, 0)

    points = [
        (1, 0.5, -50),
        (1, 0.5, 0),
        (1, 0., -50),
        (0, 0.5, -50),
    ]

    for point in points:
        numpy.testing.assert_allclose(
            func_check(*point), wgs_to_ecef_ref(*point))

    rng = numpy.random.Generator(numpy.random.Philox(1234))

    for _ in range(10**3):
        lat = numpy.arcsin(rng.uniform(-1, 1)) * RAD_TO_DEG
        lng = rng.uniform(-180, 180)
        alt = rng.uniform(-1e4, 1e4)
        numpy.testing.assert_allclose(
            func_check(lat, lng, alt), wgs_to_ecef_ref(lat, lng, alt))

    # check vector-scalar equality
    n = 10**3
    lats = numpy.arcsin(rng.uniform(-1, 1, size=n)) * RAD_TO_DEG
    lngs = rng.uniform(-180, 180, size=n)
    alts = rng.uniform(-1e4, 1e4, size=n)
    xs, ys, zs = func_check(lats, lngs, alts)

    for x, y, z, lat, lng, alt in zip(xs, ys, zs, lats, lngs, alts):
        numpy.testing.assert_array_equal((x, y, z), func_check(lat, lng, alt))


def check_both(ecef_to_wgs_check, wgs_to_ecef_check):
    rng = numpy.random.Generator(numpy.random.Philox(9))
    n = 10**7

    lat = numpy.arcsin(rng.uniform(-1, 1, size=n)) * RAD_TO_DEG
    lng = rng.uniform(-180, 180, size=n)
    # We lose accuracy for large negative altitudes, but
    # earth's crust maxes out at 2e5 meters, which I don't
    # think google are diving past just yet.
    # (according to https://en.wikipedia.org/wiki/Structure_of_Earth)
    alt = rng.uniform(-2e5, 1e7, size=n)

    # there...
    x, y, z = wgs_to_ecef_check(lat, lng, alt)
    # ... and back ...
    lat_, lng_, alt_ = ecef_to_wgs_check(x, y, z)
    # ... and back again.
    x_, y_, z_ = wgs_to_ecef_check(lat_, lng_, alt_)

    numpy.testing.assert_allclose(lat, lat_)
    numpy.testing.assert_allclose(lng, lng_)
    numpy.testing.assert_allclose(alt, alt_)

    numpy.testing.assert_allclose(x, x_)
    numpy.testing.assert_allclose(y, y_)
    numpy.testing.assert_allclose(z, z_)


def wgs_to_ecef_ref(lat, lng, alt):
    """ Reference implementation of wgs_to_ecef. """
    lat *= DEG_TO_RAD
    lng *= DEG_TO_RAD

    a = 6378137.0 # WGS-84 semi-major axis
    e2 = 6.6943799901377997e-3 # WGS-84 first eccentricity squared
    n = a / (1 - e2*numpy.sin(lat)*numpy.sin(lat))**0.5
    x = (n + alt)*numpy.cos(lat)*numpy.cos(lng)
    y = (n + alt)*numpy.cos(lat)*numpy.sin(lng)
    z = (n*(1 - e2) + alt) * numpy.sin(lat)
    return x, y, z


def ecef_to_wgs_ref(x, y, z):
    """ Reference implementation of ecef_to_wgs. """
    a = 6378137.0 # WGS-84 semi-major axis
    e2 = 6.6943799901377997e-3 # WGS-84 first eccentricity squared
    a1 = 4.2697672707157535e+4 # a1 = a*e2
    a2 = 1.8230912546075455e+9 # a2 = a1*a1
    a3 = 1.4291722289812413e+2 # a3 = a1*e2/2
    a4 = 4.5577281365188637e+9 # a4 = 2.5*a2
    a5 = 4.2840589930055659e+4 # a5 = a1+a3
    a6 = 9.9330562000986220e-1 # a6 = 1-e2

    zp = abs(z)
    w2 = x*x + y*y
    w = math.sqrt(w2)
    r2 = w2 + z*z
    r = math.sqrt(r2)
    lng = math.atan2(y, x)
    s2 = z*z/r2
    c2 = w2/r2
    u = a2/r
    v = a3 - a4/r
    if c2 > 0.3:
        s = (zp / r)*(1.0 + c2*(a1 + u + s2*v)/r)
        lat = math.asin(s)
        ss = s*s
        c = math.sqrt(1.0 - ss)
    else:
        c = (w / r)*(1.0 - s2*(a5 - u - c2*v)/r)
        lat = math.acos(c)
        ss = 1.0 - c*c
        s = math.sqrt(ss)
    g = 1.0 - e2*ss
    rg = a/math.sqrt(g)
    rf = a6*rg
    u = w - rg*c
    v = zp - rf*s
    f = c*u + s*v
    m = c*v - s*u
    p = m/(rf/g + f)
    lat = lat + p
    alt = f + m*p*0.5
    if z < 0.0:
        lat = -lat

    lat *= RAD_TO_DEG
    lng *= RAD_TO_DEG
    return lat, lng, alt



# Benchmarking

def make_benchmark_ecef_to_wgs(n):
    rng = numpy.random.Generator(numpy.random.Philox(321))
    points = rng.normal(scale=6e6, size=(n, 3))

    def benchmark_one(ecef_to_wgs_func):
        return [ecef_to_wgs_func(*point) for point in points]

    def benchmark_vec(ecef_to_wgs_func):
        return ecef_to_wgs_func(*points.T)

    return benchmark_one, benchmark_vec


def make_benchmark_wgs_to_ecef(n):
    rng = numpy.random.Generator(numpy.random.Philox(321))

    lat = numpy.arcsin(rng.uniform(-1, 1, size=n)) * RAD_TO_DEG
    lng = rng.uniform(-180, 180, size=n)
    alt = rng.uniform(-1e4, 1e4, size=n)

    points = numpy.stack([lat, lng, alt])

    def benchmark_one(wgs_to_ecef_func):
        return [wgs_to_ecef_func(*point) for point in points.T]

    def benchmark_vec(wgs_to_ecef_func):
        return wgs_to_ecef_func(*points)

    return benchmark_one, benchmark_vec

Writing wgs_ecef.py


In [2]:
import os
import numpy
import pandas
import sys
sys.path.append('./')
import wgs_ecef

import pandas as pd
def blend(folder_to_weight):
    """ Return a weighted mean of folders' submissions. 
    
        Assumes equal ordering to submission files' rows.
    """
    norm = sum(folder_to_weight.values())

    def submissions():
        for csv_dir, weight in folder_to_weight.items():
            frame = pandas.read_csv(
                csv_dir,
                dtype={
                    "tripId": str,
                    "UnixTimeMillis": numpy.uint64,
                    "LatitudeDegrees": numpy.float64,
                    "LongitudeDegrees": numpy.float64,
                })
            yield frame, weight / norm

    subs = submissions()

    # add in ECEF coordinates at 0 altitude
    example, weight = next(subs)
    xyz = get_ecef(example) * weight

    for frame, weight in subs:
        xyz += get_ecef(frame) * weight

    # convert back to WSG, update example in place
    example.LatitudeDegrees, example.LongitudeDegrees, _ = wgs_ecef.ecef_to_wgs(*xyz)

    return example


def get_ecef(frame):
    """ Return ECEF positions from frame WGS at sea level. """
    lat = frame.LatitudeDegrees
    lng = frame.LongitudeDegrees
    alt = numpy.zeros_like(frame.LatitudeDegrees)
    return numpy.stack(wgs_ecef.wgs_to_ecef(lat, lng, alt))

In [3]:
submission = blend({
    "/content/submit_cv2_90_new.csv": 0.7,
    "/content/submission_public_nb_lb300.csv": 0.3,
})

In [4]:
sub = pd.read_csv('/content/sample_submission.csv')
sub = sub.assign(LatitudeDegrees=submission.LatitudeDegrees, LongitudeDegrees=submission.LongitudeDegrees)

In [5]:
sub.to_csv("submit_for_valid.csv", index=False)

In [ ]:
'''submission = blend({
    "/content/submit_cv2_95.csv": 0.2,
    "/content/submit_cv365.csv": 0.2,
    "/content/submit_cv3_00.csv": 0.2,
    "/content/submit_cv3_11.csv": 0.2,
    "/content/submit_cv3_30.csv": 0.2,
})'''
submission = blend({
    "/content/submission_public_nb_lb300.csv": 0.3,
    "/content/submit_cv365.csv": 0.3,
    "/content/submit_cv3_30.csv": 0.4,
})

In [ ]:
submission = blend({
    "/content/submit_cv3_11.csv": 0.25,
    "/content/submit_cv365.csv": 0.25,
    "/content/submit_lb267.csv": 0.25,
    "/content/submit_cv3_30.csv": 0.25,

})

In [ ]:
submission = blend({
    "/content/submission_public_nb_lb300.csv": 0.3,
    "/content/submit_cv365.csv": 0.3,
    "/content/submit_lb267.csv": 0.4,
})

In [ ]:
submission = blend({
    "/content/submit_cv2_92.csv": 0.2,
    "/content/submit_cv2_95.csv": 0.2,
    "/content/submit_cv3_00.csv": 0.15,
    "/content/submit_cv3_05.csv": 0.15,
    "/content/submit_cv3_11.csv": 0.15,
    "/content/submit_cv3_30.csv": 0.15,
})

In [ ]:
submission = blend({
    '/content/submit_cv2_87.csv': 0.175,
    "/content/submit_cv2_92.csv": 0.125,
    "/content/submit_cv2_923.csv": 0.125,
    "/content/submit_cv2_95.csv": 0.1,
    "/content/submit_cv3_00.csv": 0.1,
    "/content/submit_cv3_05.csv": 0.1,
    "/content/submit_cv3_11.csv": 0.1,
    "/content/submit_cv2_90.csv": 0.175,
})

In [ ]:
submission = blend({
    '/content/submit_cv2_87.csv': 0.2,
    "/content/submit_cv2_92.csv": 0.125,
    "/content/submit_cv2_923.csv": 0.125,
    "/content/submit_cv2_95.csv": 0.1,
    "/content/submit_cv3_00.csv": 0.1,
    "/content/submit_cv3_05.csv": 0.1,
    "/content/submit_cv3_11.csv": 0.075,
    "/content/submit_cv2_90.csv": 0.175,
})

In [ ]:
submission = blend({
    '/content/submit_cv2_87.csv': 0.125,
    "/content/submit_cv2_92.csv": 0.125,
    "/content/submit_cv2_923.csv": 0.125,
    "/content/submit_cv2_95.csv": 0.125,
    "/content/submit_cv3_00.csv": 0.125,
    "/content/submit_cv3_05.csv": 0.125,
    "/content/submit_cv3_11.csv": 0.125,
    "/content/submit_cv2_90.csv": 0.125,
})

In [ ]:
submission = blend({
    '/content/submit_cv2_87.csv': 0.2,
    "/content/submit_cv2_92.csv": 0.2,
    "/content/submit_cv2_923.csv": 0.1,
    "/content/submit_cv2_95.csv": 0.1,
    "/content/submit_cv3_00.csv": 0.1,
    "/content/submit_cv3_05.csv": 0.1,
    "/content/submit_cv3_11.csv": 0.1,
    "/content/submit_cv3_30.csv": 0.1,
})

In [ ]:
submission = blend({
    "/content/cauchy_test_CauchySoftforXiaomi_Softfor4_lb268.csv": 0.3,
    "/content/rprtkv4_replace_outliers_th10.csv": 0.2,
    "/content/submit_final_rtks.csv": 0.5,
})

In [ ]:
submission = blend({
    "/content/cauchy_test_CauchySoftforXiaomi_Softfor4_lb268.csv": 0.3,
    "/content/rprtkv4_replace_outliers_th10.csv": 0.1,
    "/content/submit_final_rtks_v2.csv": 0.6,
})

In [ ]:
submission = blend({
    "/content/cauchy_test_CauchySoftforXiaomi_Softfor4_lb268.csv": 0.3,
    "/content/rprtkv4_replace_outliers_th10.csv": 0.1,
    "/content/submit_final_rtks_v3.csv": 0.6,
})

In [ ]:
submission = blend({
    "/content/cauchy_test_CauchySoftforXiaomi_Softfor4_lb268.csv": 0.35,
    "/content/rprtkv4_replace_outliers_th10.csv": 0.1,
    "/content/submit_final_rtks_v3.csv": 0.55,
})

In [ ]:
submission = blend({
    "/content/cauchy_test_CauchySoftforXiaomi_Softfor4_lb268.csv": 0.3,
    "/content/rprtkv4_replace_outliers_th10.csv": 0.2,
    "/content/submit_final_rtks.csv": 0.5,
})

In [ ]:
submission = blend({
    "/content/cauchy_test_CauchySoftforXiaomi_Softfor4_lb268.csv": 0.3,
    "/content/rprtkv4_replace_outliers_th10.csv": 0.3,
    "/content/submission_8_rtk_0728.csv": 0.4,
})

In [ ]:
sub = pd.read_csv('/content/sample_submission.csv')
sub = sub.assign(LatitudeDegrees=submission.LatitudeDegrees, LongitudeDegrees=submission.LongitudeDegrees)

In [ ]:
sub.to_csv("submission_ensemble_v7_0729.csv", index=False)

In [ ]:
sub.to_csv("submission_ensemble_v6_0729.csv", index=False)

In [ ]:
sub.to_csv("submission_ensemble_v5_0729.csv", index=False)

In [ ]:
sub.to_csv("submit_final_rtks_v3.csv", index=False)

In [ ]:
sub.to_csv("submission_ensemble_v4_0729.csv", index=False)

In [ ]:
sub.to_csv("submit_final_rtks_v2.csv", index=False)

In [ ]:
sub.to_csv("submission_ensemble_v3_0729.csv", index=False)

In [ ]:
sub.to_csv("submit_final_rtks.csv", index=False)

In [ ]:
sub.to_csv("submission_wls_rtk_0729.csv", index=False, )

In [ ]:
sub.to_csv("submission_ensemble_v2_0728.csv", index=False, )

In [ ]:
sub.to_csv("submission_8_rtk_0728.csv", index=False, encoding='utf-8')

In [ ]:
sub.to_csv("submission_4_models_rtk_0727.csv", index=False)

In [ ]:
sub.to_csv("submission_all_rtk.csv", index=False)

In [ ]:
sub.to_csv("submission_3models_330_365_lb300.csv", index=False)

In [ ]:
sub.to_csv("submission_3models_lb267_365_lb300_20220726.csv", index=False)

In [ ]:
ls

__pycache__/                               submission_0716.csv
rtklib_pp_by_RP_new_hypers_v3_lb2_478.csv  submission_public_nb_lb300.csv
sample_data/                               wgs_ecef.py
sample_submission.csv
